# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [ ]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [ ]:
ruta = "/content/dataset_banco_clean.csv"
data = pd.read_csv(ruta)

In [ ]:
print(data.shape)
data.head()

(45189, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

## Aplicar las técnicas oportunas de procesamiento de datos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Dividir el conjunto de datos en características y variable objetivo
X = data.drop(columns=['y'])  # Características
y = data['y']  # Variable objetivo

# Definir las transformaciones para características numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combinar las transformaciones en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Aplicar preprocesamiento y dividir en conjuntos de entrenamiento y prueba
X_preprocessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Verificar las dimensiones después del preprocesamiento
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)


Dimensiones de X_train: (36151, 51)
Dimensiones de X_test: (9038, 51)


## Generar split de los datos

In [ ]:
from sklearn.model_selection import train_test_split

# Generar split de los datos
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Verificar las dimensiones después de la división
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)


Dimensiones de X_train: (36151, 51)
Dimensiones de X_test: (9038, 51)
Dimensiones de y_train: (36151,)
Dimensiones de y_test: (9038,)


## Valorar y comparar diferentes modelos de clasificación

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score

# Definir los modelos
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "kNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": GaussianNB()
}

# Codificar etiquetas categóricas en valores numéricos
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

# Entrenar y evaluar los modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_encoded = label_encoder.transform(y_pred)
    accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
    roc_auc = roc_auc_score(y_test_encoded, y_pred_encoded)
    print(f"Modelo: {name}")
    print(f"Precisión: {accuracy:.2f}")
    print(f"AUC-ROC: {roc_auc:.2f}")
    print("="*30)


Modelo: Decision Tree
Precisión: 0.88
AUC-ROC: 0.70
Modelo: kNN
Precisión: 0.90
AUC-ROC: 0.65
Modelo: SVM
Precisión: 0.90
AUC-ROC: 0.66
Modelo: Random Forest
Precisión: 0.91
AUC-ROC: 0.69
Modelo: Naive Bayes
Precisión: 0.85
AUC-ROC: 0.71


## Aplicación de un ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Definir los modelos base
modelos_base = [
    ("Decision Tree", DecisionTreeClassifier()),
    ("kNN", KNeighborsClassifier()),
    ("SVM", SVC()),
    ("Random Forest", RandomForestClassifier()),
    ("Naive Bayes", GaussianNB())
]

# Definir el ensamble
ensamble = VotingClassifier(estimators=modelos_base)

# Codificar etiquetas categóricas en valores numéricos
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

# Entrenar y evaluar el ensamble
ensamble.fit(X_train, y_train)
y_pred_ensamblaje = ensamble.predict(X_test)

# Codificar las predicciones de cadenas a valores numéricos
y_pred_encoded = label_encoder.transform(y_pred_ensamblaje)

# Calcular las métricas
accuracy_ensamblaje = accuracy_score(y_test_encoded, y_pred_encoded)
roc_auc_ensamblaje = roc_auc_score(y_test_encoded, y_pred_encoded)

# Imprimir las métricas
print("Ensamblaje:")
print(f"Precisión: {accuracy_ensamblaje:.2f}")
print(f"AUC-ROC: {roc_auc_ensamblaje:.2f}")



Ensamblaje:
Precisión: 0.91
AUC-ROC: 0.68


## Obtener varias métricas

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Calcular métricas para los modelos individuales
for name, model in models.items():
    y_pred = model.predict(X_test)

    # Calcular métricas para el modelo actual
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='yes')
    recall = recall_score(y_test, y_pred, pos_label='yes')
    f1 = f1_score(y_test, y_pred, pos_label='yes')
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Imprimir métricas para el modelo actual
    print(f"Modelo: {name}")
    print(f"Precisión: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")
    print(f"Matriz de confusión:\n{conf_matrix}")
    print("="*30)

# Calcular métricas para el ensamblaje
y_pred_ensamblaje = ensamblaje.predict(X_test)

# Codificar etiquetas categóricas en valores numéricos
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_pred_ensamblaje_encoded = label_encoder.transform(y_pred_ensamblaje)

# Calcular métricas para el ensamblaje
accuracy_ensamblaje = accuracy_score(y_test_encoded, y_pred_ensamblaje_encoded)
precision_ensamblaje = precision_score(y_test_encoded, y_pred_ensamblaje_encoded)
recall_ensamblaje = recall_score(y_test_encoded, y_pred_ensamblaje_encoded)
f1_ensamblaje = f1_score(y_test_encoded, y_pred_ensamblaje_encoded)
conf_matrix_ensamblaje = confusion_matrix(y_test_encoded, y_pred_ensamblaje_encoded)

# Imprimir métricas del ensamblaje
print("Ensamblaje:")
print(f"Precisión: {precision_ensamblaje:.2f}")
print(f"Recall: {recall_ensamblaje:.2f}")
print(f"F1-score: {f1_ensamblaje:.2f}")
print(f"Matriz de confusión:\n{conf_matrix_ensamblaje}")



Modelo: Decision Tree
Precisión: 0.48
Recall: 0.47
F1-score: 0.47
Matriz de confusión:
[[7433  545]
 [ 561  499]]
Modelo: kNN
Precisión: 0.60
Recall: 0.33
F1-score: 0.43
Matriz de confusión:
[[7745  233]
 [ 708  352]]
Modelo: SVM
Precisión: 0.69
Recall: 0.33
F1-score: 0.45
Matriz de confusión:
[[7823  155]
 [ 707  353]]
Modelo: Random Forest
Precisión: 0.68
Recall: 0.39
F1-score: 0.50
Matriz de confusión:
[[7784  194]
 [ 642  418]]
Modelo: Naive Bayes
Precisión: 0.40
Recall: 0.52
F1-score: 0.46
Matriz de confusión:
[[7161  817]
 [ 504  556]]
Ensamblaje:
Precisión: 0.67
Recall: 0.38
F1-score: 0.49
Matriz de confusión:
[[7782  196]
 [ 654  406]]


# Conclusiones


En este trabajo, se han evaluado varios modelos de aprendizaje automático para clasificar clientes interesados en adquirir un certificado de depósito a término. Para ello, hemos cargado y procesado un dataset, separando las características (X) y la variable objetivo (y), y aplicando técnicas como imputación y escalado para las características numéricas y codificación one-hot para las categóricas. Hemos dividido los datos en conjuntos de entrenamiento y prueba, obteniendo 36,151 muestras para entrenamiento y 9,038 para prueba. Hemos evaluado diversos modelos de clasificación: Decision Tree, kNN, SVM, Random Forest y Naive Bayes.

Los resultados han mostrado que el Random Forest y el SVM han tenido las mejores precisiones (0.91), aunque su AUC-ROC no ha sido tan alto (0.69 y 0.66, respectivamente). El modelo ensemble, que ha combinado estos clasificadores, ha mantenido una precisión de 0.91 y un AUC-ROC de 0.68, indicando una mejora general en la robustez del modelo.

Las métricas detalladas recall y F1-score, han mostrado diferencias significativas: por ejemplo, el Decision Tree ha tenido una precisión de 0.48 y un F1-score de 0.47, mientras que el ensemble ha alcanzado una precisión de 0.67 y un F1-score de 0.49. La matriz de confusión del ensemble muestra que este modelo equilibra mejor las predicciones correctas y los errores, confirmando la utilidad de combinar varios modelos para mejorar el rendimiento en tareas de clasificación complejas.